In [15]:
import math
from geopy.geocoders import Nominatim

def geocoding(address):
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    geo = geolocoder.geocode(address)
    crd = {"lat": str(geo.latitude), "lng": str(geo.longitude)}

    return crd

crd = geocoding(input("위치를 입력해주세요: "))
lat = int(float(crd['lat']))
lng = int(float(crd['lng']))


NX = 149            ## X축 격자점 수
NY = 253            ## Y축 격자점 수

Re = 6371.00877     ##  지도반경
grid = 5.0          ##  격자간격 (km)
slat1 = 30.0        ##  표준위도 1
slat2 = 60.0        ##  표준위도 2
olon = 126.0        ##  기준점 경도
olat = 38.0         ##  기준점 위도
xo = 210 / grid     ##  기준점 X좌표
yo = 675 / grid     ##  기준점 Y좌표
first = 0

if first == 0 :
    PI = math.asin(1.0) * 2.0
    DEGRAD = PI/ 180.0
    RADDEG = 180.0 / PI


    re = Re / grid
    slat1 = slat1 * DEGRAD
    slat2 = slat2 * DEGRAD
    olon = olon * DEGRAD
    olat = olat * DEGRAD

    sn = math.tan(PI * 0.25 + slat2 * 0.5) / math.tan(PI * 0.25 + slat1 * 0.5)
    sn = math.log(math.cos(slat1) / math.cos(slat2)) / math.log(sn)
    sf = math.tan(PI * 0.25 + slat1 * 0.5)
    sf = math.pow(sf, sn) * math.cos(slat1) / sn
    ro = math.tan(PI * 0.25 + olat * 0.5)
    ro = re * sf / math.pow(ro, sn)
    first = 1

def mapToGrid(lat, lng, code = 0 ):
    ra = math.tan(PI * 0.25 + lat * DEGRAD * 0.5)
    ra = re * sf / pow(ra, sn)
    theta = lng * DEGRAD - olon
    if theta > PI :
        theta -= 2.0 * PI
    if theta < -PI :
        theta += 2.0 * PI
    theta *= sn
    x = (ra * math.sin(theta)) + xo
    y = (ro - ra * math.cos(theta)) + yo
    x = int(x + 1.5)
    y = int(y + 1.5)
    return x, y

lat, lng = mapToGrid(lat, lng)

In [14]:
import requests
from datetime import datetime, date, timedelta
from urllib import parse
from urllib.parse import quote_plus

url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'
params ={'serviceKey' : '서비스키', 'pageNo' : '1', 'numOfRows' : '1000', 'dataType' : 'XML', 'base_date' : '20210628', 'base_time' : '0600', 'nx' : '55', 'ny' : '127' }

response = requests.get(url, params=params)
print(response.content)

# 기상청_동네 예보 조회 서비스 api 데이터 url 주소, 초단기이기때문에 getUltraSrtFcst 사용
url = "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst"

serviceKey = "%2FbubrDTNA1tkeqcpHxeE3GIcZwAzdDVQqN23NbkoUsySRKCO1MsLOk3Dv3c9Fuu%2F9XcYHYvfe1E7rEk4joZh8A%3D%3D" # 공공데이터 포털에서 생성된 본인의 서비스 키를 복사 / 붙여넣기
# serviceKeyDecoded = unquote(serviceKey, 'UTF-8') # 공데이터 포털에서 제공하는 서비스키는 이미 인코딩된 상태이므로, 디코딩하여 사용해야 함
serviceKeyDecoded = "/bubrDTNA1tkeqcpHxeE3GIcZwAzdDVQqN23NbkoUsySRKCO1MsLOk3Dv3c9Fuu/9XcYHYvfe1E7rEk4joZh8A==" # 공데이터 포털에서 제공하는 서비스키는 이미 인코딩된 상태이므로, 디코딩하여 사용해야 함

now = datetime.now()
today = datetime.today().strftime("%Y%m%d")
y = date.today() - timedelta(days=1)
yesterday = y.strftime("%Y%m%d")

# 위도와 경도를 x,y좌표로 변경한 값 위에서 가져온다
nx = lat 
ny = lng

if now.minute<45: # base_time와 base_date 구하는 함수
    if now.hour==0:
        base_time = "2330"
        base_date = yesterday
    else:
        pre_hour = now.hour-1
        if pre_hour<10:
            base_time = "0" + str(pre_hour) + "30"
        else:
            base_time = str(pre_hour) + "30"
        base_date = today
else:
    if now.hour < 10:
        base_time = "0" + str(now.hour) + "30"
    else:
        base_time = str(now.hour) + "30"
    base_date = today



queryParams = '?' + parse.urlencode({ quote_plus('serviceKey') : serviceKeyDecoded, quote_plus('base_date') : base_date,
                                quote_plus('base_time') : base_time, quote_plus('nx') : nx, quote_plus('ny') : ny,
                                quote_plus('dataType') : 'json', quote_plus('numOfRows') : '60'}) #페이지로 안나누고 한번에 받아오기 위해 numOfRows=60으로 설정해주었다
                                

# 값 요청 (웹 브라우저 서버에서 요청 - url주소와 파라미터)
res = requests.get(url + queryParams, verify=False) # verify=False이거 안 넣으면 에러남
items = res.json().get('response').get('body').get('items') #데이터들 아이템에 저장
#print(items)# 테스트

# weather_data = dict()

# for item in items['item']:
#     # 기온
#     if item['category'] == 'T1H':
#         weather_data['tmp'] = item['fcstValue']
#     # 습도
#     if item['category'] == 'REH':
#         weather_data['hum'] = item['fcstValue']
#     # 하늘상태: 맑음(1) 구름많은(3) 흐림(4)
#     if item['category'] == 'SKY':
#         weather_data['sky'] = item['fcstValue']
#     # 1시간 동안 강수량
#     if item['category'] == 'RN1':
#         weather_data['rain'] = item['fcstValue']

# print("response: ", weather_data)

def weather_forecast():
    
    weather_data = dict()

    for item in items['item']:
        # 기온
        if item['category'] == 'T1H':
            weather_data['tmp'] = item['fcstValue']
        # 습도
        if item['category'] == 'REH':
            weather_data['hum'] = item['fcstValue']
        # 하늘상태: 맑음(1) 구름많은(3) 흐림(4)
        if item['category'] == 'SKY':
            weather_data['sky'] = item['fcstValue']
        # 1시간 동안 강수량
        if item['category'] == 'RN1':
            weather_data['rain'] = item['fcstValue']
    
    return weather_data

def proc_weather():
    dict_sky = weather_forecast()

    str_sky = "현재 "
    if dict_sky['sky'] != None:
        str_sky = str_sky + "날씨 : "
        
        if dict_sky['sky'] == '1':
            str_sky = str_sky + "맑음"
        elif dict_sky['sky'] == '3':
            str_sky = str_sky + "구름많음"
        elif dict_sky['sky'] == '4':
            str_sky = str_sky + "흐림"
            
        str_sky = str_sky + "\n"
        
    if dict_sky['tmp'] != None:
        str_sky = str_sky + "온도 : " + dict_sky['tmp'] + 'ºC \n'
    if dict_sky['hum'] != None:
        str_sky = str_sky + "습도 : " + dict_sky['hum'] + '%'

    return str_sky
    
print(proc_weather())

b'<OpenAPI_ServiceResponse>\n\t<cmmMsgHeader>\n\t\t<errMsg>SERVICE ERROR</errMsg>\n\t\t<returnAuthMsg>SERVICE_KEY_IS_NOT_REGISTERED_ERROR</returnAuthMsg>\n\t\t<returnReasonCode>30</returnReasonCode>\n\t</cmmMsgHeader>\n</OpenAPI_ServiceResponse>'
현재 날씨 : 구름많음
온도 : 21ºC 
습도 : 95%


/Users/hedge/miniconda3/envs/newenv/lib/python3.11/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


---

- 위치 입력하면 자동으로 경도/위도 구하기

In [22]:
# geopy는 느리지만 별도의 api 키 발급 등이 필요 없음 / 대신, 위도 경도가 살짝 다른것 같음.
# 다른 방법은 구글맵 api 활용법임

# !pip install geopy

  Obtaining dependency information for geopy from https://files.pythonhosted.org/packages/e1/58/9289c6a03116025cdb61461d99b2493daa4967a80b13755463d71a0affeb/geopy-2.4.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 337.1 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 623.8 kB/s eta 0:00:0000:0100:01


In [8]:
# 테스트

from geopy.geocoders import Nominatim

def geocoding(address):
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    geo = geolocoder.geocode(address)
    crd = {"lat": str(geo.latitude), "lng": str(geo.longitude)}

    return crd

crd = geocoding(input("위치를 입력해주세요: "))
print(crd['lat'])
print(crd['lng'])

lat = crd['lat']
lng = crd['lng']


37.4964975
127.0698841


In [12]:
# 위도, 경도를 x, y 좌표로 바꾸기

import math
from geopy.geocoders import Nominatim


def geocoding(address):
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    geo = geolocoder.geocode(address)
    crd = {"lat": str(geo.latitude), "lng": str(geo.longitude)}

    return crd

crd = geocoding(input("위치를 입력해주세요: "))
lat = int(float(crd['lat']))
lng = int(float(crd['lng']))


NX = 149            ## X축 격자점 수
NY = 253            ## Y축 격자점 수

Re = 6371.00877     ##  지도반경
grid = 5.0          ##  격자간격 (km)
slat1 = 30.0        ##  표준위도 1
slat2 = 60.0        ##  표준위도 2
olon = 126.0        ##  기준점 경도
olat = 38.0         ##  기준점 위도
xo = 210 / grid     ##  기준점 X좌표
yo = 675 / grid     ##  기준점 Y좌표
first = 0

if first == 0 :
    PI = math.asin(1.0) * 2.0
    DEGRAD = PI/ 180.0
    RADDEG = 180.0 / PI


    re = Re / grid
    slat1 = slat1 * DEGRAD
    slat2 = slat2 * DEGRAD
    olon = olon * DEGRAD
    olat = olat * DEGRAD

    sn = math.tan(PI * 0.25 + slat2 * 0.5) / math.tan(PI * 0.25 + slat1 * 0.5)
    sn = math.log(math.cos(slat1) / math.cos(slat2)) / math.log(sn)
    sf = math.tan(PI * 0.25 + slat1 * 0.5)
    sf = math.pow(sf, sn) * math.cos(slat1) / sn
    ro = math.tan(PI * 0.25 + olat * 0.5)
    ro = re * sf / math.pow(ro, sn)
    first = 1

def mapToGrid(lat, lng, code = 0 ):
    ra = math.tan(PI * 0.25 + lat * DEGRAD * 0.5)
    ra = re * sf / pow(ra, sn)
    theta = lng * DEGRAD - olon
    if theta > PI :
        theta -= 2.0 * PI
    if theta < -PI :
        theta += 2.0 * PI
    theta *= sn
    x = (ra * math.sin(theta)) + xo
    y = (ro - ra * math.cos(theta)) + yo
    x = int(x + 1.5)
    y = int(y + 1.5)
    return x, y

# def gridToMap(x, y, code = 1):
#     x = x - 1
#     y = y - 1
#     xn = x - xo
#     yn = ro - y + yo
#     ra = math.sqrt(xn * xn + yn * yn)
#     if sn < 0.0 :
#         ra = -ra
#     alat = math.pow((re * sf / ra), (1.0 / sn))
#     alat = 2.0 * math.atan(alat) - PI * 0.5
#     if math.fabs(xn) <= 0.0 :
#         theta = 0.0
#     else :
#         if math.fabs(yn) <= 0.0 :
#             theta = PI * 0.5
#             if xn < 0.0 :
#                 theta = -theta
#         else :
#             theta = math.atan2(xn, yn)
#     alon = theta / sn + olon
#     lat = alat * RADDEG
#     lng = alon * RADDEG

#     return lat, lng

print(mapToGrid(lat, lng))
# print(mapToGrid(37.579871128849334, 126.98935225645432))
# print(mapToGrid(35.101148844565955, 129.02478725562108))
# print(mapToGrid(33.500946412305076, 126.54663058817043))
# ### result :
# #(60, 127)
# #(97, 74)
# #(53, 38)
#
# print(gridToMap(60, 127))
# print(gridToMap(97, 74))
# print(gridToMap(53, 38))
### result
# 37.579871128849334, 126.98935225645432
# 35.101148844565955, 129.02478725562108
# 33.500946412305076, 126.54663058817043

(60, 114)
